# HR Policy and Tool-Calling Agent

## Overview
AI agent using tool-calling to analyze HR policies and evaluate employee promotion eligibility. Processes employee data from **SAP SuccessFactors Data Products** to provide actionable insights.

## Key Components

### 1. Tool-Calling Architecture
- **Purpose**: Enables the AI to dynamically select and execute appropriate tools/functions
- **Benefits**: 
  - More accurate responses by accessing real-time data
  - Modular and extensible design
  - Better control over AI behavior

### 2. HR Policy Integration
- **Policy Database**: Centralized repository of company HR policies
- **Query Interface**: Natural language interface for policy questions
- **Context Awareness**: Agent understands employee context and role-specific policies

### 3. Agent Workflow
1. **Question Parsing**: Understands user intent and required information
2. **Tool Selection**: Chooses appropriate tools (policy lookup, employee data, etc.)
3. **Execution**: Runs selected tools with proper parameters
4. **Response Generation**: Synthesizes results into clear, actionable answers

## Use Cases
- Policy clarification requests
- Career path recommendations
- Compliance verification
- Employee eligibility checks
- Benefits and leave policy queries

## Setup
Install required packages and import libraries for PDF parsing and Spark operations.


In [0]:
# Install the correct package in a notebook cell
%pip install pymupdf

from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import pymupdf as fitz  # PyMuPDF for PDF parsing

## Data Loading
Load employee and performance data from SAP SuccessFactors Data Products, then transform and enrich the dataset with performance ratings.


In [0]:
# Load data from SAP SuccessFactors Data Products
print("📊 Loading SAP SuccessFactors BDC data from Data Products...")

# Load from SAP SuccessFactors Data Products using spark.sql (fast Databricks loading)
employees_df_raw = spark.sql(
    "SELECT * FROM core_workforce_data_dp.coreworkforcedata.coreworkforce_standardfields"
)

performance_df_raw = spark.sql(
    "SELECT * FROM performance_reviews_dp.performancereviews.commentfeedback"
)

# Map columns to expected names (keep original structure, just add aliases)
employees_df = employees_df_raw.select(
    F.col("userId").alias("employee_id"),
    F.col("age"),
    F.col("gender"),
    F.col("department"),
    F.col("jobTitle").alias("job_title"),
    F.when(F.col("jobTitle").rlike("Manager|Director|VP|Chief"), 3)
     .when(F.col("jobTitle").rlike("Senior|Lead|Principal|Staff"), 2)
     .otherwise(1).alias("job_level"),
    F.col("location"),
    F.col("employmentType").alias("employment_type"),
    F.col("annualSalary").alias("base_salary"),
    (F.col("totalOrgTenureCalc") / 30).cast("int").alias("tenure_months"),
    (F.col("totalPositionTenureCalc") / 30).cast("int").alias("months_in_current_role"),
    F.col("employmentStatus").alias("employment_status"),
    F.lit("Unknown").alias("first_name"),  # Add default if not in source
    F.lit("Unknown").alias("last_name")    # Add default if not in source
)

# Process performance data
performance_df = performance_df_raw.select(
    F.col("subject_8995a2862a8343bd8390aaa82c46e881").alias("employee_id"),
    F.col("modifiedAt").alias("review_date"),
    F.col("numberValue").alias("overall_rating"),
    F.col("numberValue").alias("competency_rating"),
    (F.col("numberValue") * 20).cast("int").alias("goals_achievement"),
    F.col("id").cast("string").alias("review_id")
).filter(F.col("numberValue").isNotNull() & (F.col("numberValue") > 0))

# Get latest performance rating for each employee
latest_performance = performance_df.withColumn(
    "row_num",
    F.row_number().over(
        Window.partitionBy("employee_id")
        .orderBy(F.col("review_date").desc())
    )
).filter(F.col("row_num") == 1).drop("row_num")

# Join employees with latest performance data to create enriched dataset
df_employees = employees_df.alias("e").join(
    latest_performance.alias("p"),
    F.col("e.employee_id") == F.col("p.employee_id"),
    "left"
).select(
    F.col("e.employee_id").alias("EmployeeID"),
    F.col("e.first_name").alias("Name"),  # Use first_name from employees_df
    F.col("e.department").alias("Department"),
    F.col("e.months_in_current_role").alias("YearsInDepartment"),  # Using months as years for compatibility
    F.coalesce(F.col("p.overall_rating") * 20, F.lit(70)).alias("PerformanceScore"),  # Convert rating (1-5) to score (20-100)
    F.lit(False).alias("AlreadyPromoted")  # Default to False, can be updated based on actual data
)

# Save as a permanent Delta table in Unity Catalog
df_employees.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.default.employee_details")

print(f"✅ Data loaded: {df_employees.count():,} employees")

## Policy Document Extraction
Extract text from the HR promotion policy PDF document for analysis.


In [0]:
# save employee dataframe as a table (for backward compatibility with existing queries)
df_employees.write.mode("overwrite").saveAsTable("default.employee_data")

## Policy Rule Extraction
Use LLM to parse the policy document and extract structured promotion rules (minimum months, performance cycles, exceptions) as JSON.


In [0]:
# UPDATE PATH TO WHERE THE POLICY PDF TEXT IS STORED
pdf_path = "/Workspace/Users/jing.wen.wang@sap.com/HR Policy into Actionable Promotion Insights/Employee-Promotion-SOP-Samples.pdf"  

with fitz.open(pdf_path) as doc:
    policy_text = ""
    for page in doc:
        policy_text += page.get_text()

# Optionally, preview the extracted text
print(policy_text[:1000])  # Show first 1000 characters

## Save Policy Rules
Store extracted policy rules in a Spark table for programmatic access.


In [0]:
# --- Section 3: Ask LLM for JSON Directly ---
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"

prompt = f"""
You are an AI assistant specializing in HR promotion policy analysis.

Read the promotion policy below and extract these values into a JSON object:
- min_months_in_position (integer, months required in department before promotion)
- min_performance_cycles (integer, number of recent review cycles with high performance)
- exceptions (list of strings)

If a value is not explicitly stated, set it to null or an empty list.

Return only valid JSON — no explanations, no extra text.

Policy:
\"\"\"
{policy_text}
\"\"\"
"""

safe_prompt = prompt.replace("'", "''")

sql_query = f"""
SELECT ai_query('{LLM_ENDPOINT_NAME}', '{safe_prompt}') AS promotion_policy_json
"""
result_df = spark.sql(sql_query)
raw_output = result_df.collect()[0]["promotion_policy_json"].strip()

# Clean JSON if wrapped in markdown fences
if raw_output.startswith("```"):
    raw_output = raw_output.strip("`").replace("json", "", 1).strip()

import json
try:
    policy_dict = json.loads(raw_output)
except json.JSONDecodeError:
    # Fallback in case model doesn't follow instructions
    policy_dict = {
        "min_months_in_position": None,
        "min_performance_cycles": None,
        "exceptions": []
    }

print("\n=== Parsed JSON from LLM ===\n", json.dumps(policy_dict, indent=2))

## Promotion Eligibility Function
Create a SQL function that evaluates employee promotion eligibility based on tenure, performance cycles, and performance plan status.


In [0]:
# --- Section 4: Save JSON to Spark Table ---
from pyspark.sql import Row

policy_row = Row(
    min_months_in_position = policy_dict.get("min_months_in_position") or 0,
    min_performance_cycles = policy_dict.get("min_performance_cycles") or 0,
    exceptions = json.dumps(policy_dict.get("exceptions", []))
)

policy_df = spark.createDataFrame([policy_row])
policy_df.write.mode("overwrite").saveAsTable("workspace.default.promotion_policy_rules")
display(policy_df)


min_months_in_position,min_performance_cycles,exceptions
3,2,"[""Managers' subjective opinions unsupported by performance evaluations or metrics"", ""Discrimination"", ""Fraternization"", ""Favoritism"", ""Nepotism""]"


## AI-Powered Validation Function
Create a tool-calling function that uses AI to validate employee records and identify potential issues or missing information for promotion eligibility assessment.


In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.default.is_employee_eligible_for_promotion(
  months_in_position INT,
  high_perf_cycles INT,
  under_performance_plan BOOLEAN
)
RETURNS BOOLEAN
RETURN (
  SELECT
    CASE
      WHEN months_in_position < (
        SELECT COALESCE(min_months_in_position, 0)
        FROM workspace.default.promotion_policy_rules
        LIMIT 1
      ) THEN FALSE
      WHEN high_perf_cycles < (
        SELECT COALESCE(min_performance_cycles, 0)
        FROM workspace.default.promotion_policy_rules
        LIMIT 1
      ) THEN FALSE
      WHEN under_performance_plan = TRUE THEN FALSE
      ELSE TRUE
    END
);

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.default.dox_validate_employee_promotion(employee_id STRING)
RETURNS STRING
RETURN (
  SELECT ai_query(
    'databricks-claude-3-7-sonnet',
    CONCAT(
      'You are a senior HR analyst reviewing employee promotion eligibility.\n\n',
      'Please analyze the completeness and consistency of the following employee record (ID: ', CAST(MAX(EmployeeID) AS STRING), ').\n',
      'Employee fields:\n',
      '- Name: ', COALESCE(MAX(Name), 'NULL'), '\n',
      '- Department: ', COALESCE(MAX(Department), 'NULL'), '\n',
      '- YearsInDepartment: ', COALESCE(CAST(MAX(YearsInDepartment) AS STRING), 'NULL'), '\n',
      '- PerformanceScore: ', COALESCE(CAST(MAX(PerformanceScore) AS STRING), 'NULL'), '\n',
      '- AlreadyPromoted: ', COALESCE(CAST(MAX(AlreadyPromoted) AS STRING), 'NULL'), '\n\n',
      'Instructions:\n',
      '1. Identify any missing or suspicious data fields.\n',
      '2. Highlight any potential reasons why the employee might not be eligible for promotion.\n',
      '3. Suggest additional checks or data fields that could improve the promotion eligibility assessment.\n\n',
      'Respond as 3-5 clear bullet points.'
    )
  )
  FROM workspace.default.employee_details
  WHERE EmployeeID = CAST(employee_id AS INT)
);